In [118]:
import re
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from konlpy.tag import Okt
from ckonlpy.tag import Twitter as Okt
from ckonlpy.tag import Postprocessor
from collections import Counter

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

code_path = '/VSC/JJaemni/Code/'
csv_path = '/VSC/JJaemni/CSV/'

df = pd.read_csv(csv_path + 'wanted_jobplanet_2.csv')

logging.getLogger('konlpy').setLevel(logging.ERROR)

# import matplotlib.pyplot as plt

# # 가능한 font list 확인
# import matplotlib.font_manager as fm
# f = [f.name for f in fm.fontManager.ttflist]
# print(f)

# # 확인 이후
# plt.rc('font', family='Malgun Gothic')

# **불용어 제거 및 표준화**
- 한 행 씩 보면서, 불용어 표준화 진행을 위한 전처리를 먼저 수행

## **Requirement: Kor**

### **명사 추가, 단어 치환, 불용어 처리**
- `customized_konlpy`

In [825]:
# 한글, 영문 대소문자, 숫자, 공백을 제외한 문자를 제거
req = df.Requirement.tolist()

req2 = []
for idx in range(0, df.shape[0]):
    str = req[idx]
    new_str = re.sub('\n', '', str)
    new_str2 = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', ' ', new_str)
    new_str3 = re.split(' ', new_str2)
    new_str4 = ' '.join(filter(None, new_str3))
    req2.append(new_str4)
    # print(f'{idx}. {req2[idx]}')
req2_df = pd.DataFrame({'req': req2})


# Kor, Eng 분리
eng_list = []
kor_list = []
for idx in range(0, df.shape[0]):
    text = req2_df.req[idx]
    eng_text = ' '.join(re.findall(r'[A-Za-z]+', text))
    kor_text = re.sub(r'[A-Za-z]+', '', text).strip()
    eng_list.append(eng_text)
    kor_list.append(kor_text)


# 영어 띄어쓰기 기준으로 split
eng_split_list = []
for eng in eng_list:
    spl = eng.split()
    eng_split_list.append(spl)
# for idx, eng in enumerate(eng_split_list):
    # print(f'{idx}. {eng}')


# 영어 소문자로 통일
eng_lower = []
for sub in eng_split_list:
    sublist = []
    for eng in sub:
        eng_ = eng.lower()
        sublist.append(eng_)
    eng_lower.append(sublist)
# for idx, eng in enumerate(eng_lower):
    # print(f'{idx}. {eng}')
# print(len(eng_lower))


# 텍스트 파일 불러오기
with open(code_path + 'Postprocessor_Custom_Nouns.txt', 'r', encoding='utf-8-sig') as file:
    a = file.read()
    spl = a.split(',')
    custom_nouns = list()
    for i in spl:
        spl_ = i.strip(" '")
        custom_nouns.append(spl_)
    # print(custom_nouns)

with open(code_path + 'Postprocessor_Replace.txt', 'r', encoding='utf-8-sig') as file:
    b = file.read()
    spl = b.split(',')
    replace = dict()
    for i in spl:
        key_val = i.split(':')
        key = key_val[0].strip(" '")
        val = key_val[1].strip(" '")
        replace[key] = val
    # print(replace)
    
with open(code_path + 'Postprocessor_Stopwords.txt', 'r', encoding='utf-8-sig') as file:
    c = file.read()
    spl = c.split(',')
    stopwords = set()
    for i in spl:
        spl_ = i.strip(" '")
        stopwords.add(spl_) 
    # print(stopwords)


okt = Okt()

# 명사 추가
custom_nouns = custom_nouns
for noun in custom_nouns:
    okt.add_dictionary(noun, 'Noun')

# 단어 치환
replace = replace

# 불용어 처리
stopwords = stopwords

postprocessor = Postprocessor(
    base_tagger=okt,
    passtags={'Noun'},
    replace=replace,
    stopwords=stopwords,
    # passwords=passwords,
    # ngrams=ngrams
)

results = []
for idx in range(0, df.shape[0]):
    elem = kor_list[idx]
    # print(postprocessor.pos(elem))
    post_onechar = postprocessor.pos(elem)
    result = []
    for word, tag in post_onechar:
        if len(word) > 1:
            result.append(word)
    results.append(result)
    print(f'{idx}. {result}')


# 리스트 중복 요소 제거
Req_kor_final = []
for idx in range(0, df.shape[0]):
    a = set(results[idx])
    b = list(a)
    Req_kor_final.append(b)
# print(Req_kor_final)
# print(len(Req_kor_final))


# 데이터프레임 생성
Req_kor_final_df = pd.DataFrame({'Requirement_KOR': Req_kor_final})
Req_kor_final_df.Requirement_KOR = Req_kor_final_df.Requirement_KOR.apply(lambda x: ', '.join(x))
Req_kor_final_df.to_csv(csv_path + 'Requirement_Kor.csv', index=False)

c:\Users\LG\anaconda3\envs\crawling\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


0. ['시계열 데이터', '데이터 분석', '논문 구현', '알고리즘 개발', '데이터 분석', '통계분석', '인공지능', '머신러닝']
1. ['데이터 사이언스', '블록체인', '데이터 분석', '머신러닝', '온체인 데이터', '데이터 분석']
2. ['데이터 사이언스', '실무경험', '데이터 분석', '머신러닝', '데이터 분석', '문제 정의 및 해결', '알고리즘 개발', '논문 구현', '데이터 분석', '블록체인', '실무경험']
3. ['상용 서비스 구현', '데이터 모델링', '개발역량', '데이터 분석', '데이터 분석', '고객 분석', '데이터 분석', '인사이트 도출']
4. ['데이터 모델링']
5. ['딥러닝', '프레임워크', '모델 개발', '모델 개발 및 배포', '데이터 분석', '데이터 시각화', '적극적', '커뮤니케이션', '주도적', '개발경험', '협업', '프로젝트 경험', '협업', '포트폴리오']
6. ['데이터 사이언티스트', '데이터 모델링', '프로젝트 경험', '실무경험', '개인화 추천', '고객 분석', '데이터 모델링', '고객 분석', '도메인 지식', '프로그래밍 언어', '수리적 지식', '협업', '커뮤니케이션']
7. ['통계분석', '컴퓨터공학', '산업공학', '도메인 지식', '데이터 모델링', '알고리즘 개발', '자연어 처리', '모델 개발', '가설 수립 및 검증', '최적화', '통계분석', '데이터 분석', '데이터 분석', '데이터 모델링']
8. ['프로젝트 경험', '실무경험', '모바일 데이터', '데이터 분석', '머신러닝', '주도적', '끈질김', '인사이트 도출', '이슈 해결', '문제 정의 및 해결']
9. ['통계분석', '컴퓨터공학', '산업공학', '데이터 분석', '데이터 시각화', '데이터 분석', '최신 기술 연구', '열정적']
10. ['실무경험', '머신러닝', '프로젝트 경험', '리딩 능력', '논문 구현', '모델 개발', '커뮤니

## **Requirement: Eng**

In [448]:
for idx in range(0, df.shape[0]):
    print(f'{idx}. {eng_lower[idx]}')

0. ['python']
1. ['sql', 'hql', 'python', 'r']
2. ['sql', 'hql', 'python', 'r', 'complex', 'network', 'modeling']
3. ['machine', 'learnig', 'python', 'java', 'sql', 'r']
4. []
5. ['computer', 'vision', 'image', 'video', 'processing', 'tensorflow', 'pytorch', 'or']
6. ['ml', 'dl', 'python', 'sql', 'ml', 'dl']
7. ['data', 'science', 'ml', 'dl', 'nlp', 'clustering', 'ml', 'dl', 'sql', 'python']
8. ['python', 'sql', 'r']
9. ['data', 'science', 'python', 'r', 'sql', 'tool', 'splunk', 'tableau', 'tool']
10. ['ai']
11. ['python', 'data', 'analysis', 'process', 'eda', 'statistical', 'analysis', 'data', 'mining', 'report', 'deployment', 'query', 'manipulation', 'r']
12. ['python', 'sql', 'spark', 'hadoop']
13. ['ai']
14. []
15. ['python', 'r', 'ml', 'dl']
16. []
17. ['master', 's', 'degree', 'in', 'a', 'quantitative', 'discipline', 'e', 'g', 'statistics', 'operations', 'research', 'bioinformatics', 'economics', 'computational', 'biology', 'computer', 'science', 'mathematics', 'physics', 'electr

- 17행의 공고내용은 영어였음

### **불용어 처리**

In [449]:
from nltk.corpus import stopwords

a = ' '.join(eng_lower[17])
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(a)

results = []
for token in word_tokens:
    if token not in stop_words:
        results.append(token)
print(word_tokens)
print(results)

['master', 's', 'degree', 'in', 'a', 'quantitative', 'discipline', 'e', 'g', 'statistics', 'operations', 'research', 'bioinformatics', 'economics', 'computational', 'biology', 'computer', 'science', 'mathematics', 'physics', 'electrical', 'engineering', 'industrial', 'engineering', 'or', 'equivalent', 'research', 'experience', 'years', 'of', 'work', 'experience', 'in', 'data', 'science', 'related', 'fields', 'distinctive', 'problem', 'solving', 'skills', 'good', 'at', 'articulating', 'product', 'questions', 'pulling', 'data', 'from', 'large', 'datasets', 'and', 'using', 'statistics', 'to', 'arrive', 'at', 'a', 'recommendation', 'excellent', 'verbal', 'and', 'written', 'communication', 'skills', 'with', 'ability', 'to', 'present', 'information', 'and', 'analysis', 'results', 'effectively', 'ability', 'to', 'build', 'positive', 'relationships', 'within', 'dsa', 'and', 'with', 'our', 'stakeholders', 'and', 'work', 'effectively', 'with', 'cross', 'functional', 'partners', 'in', 'a', 'globa

- 영어는 일단 `StackTool` 정리한 후에 다시 처리하기

---

## **Preference: Kor**

### **명사 추가, 단어 치환, 불용어 처리**
- `customized_konlpy`

# **Preference**

In [827]:
# 한글, 영문 대소문자, 숫자, 공백을 제외한 문자를 제거
prefer = df.Preference.tolist()

prefer2 = []
for idx in range(0, df.shape[0]):
    str = prefer[idx]
    new_str = re.sub('\n', '', str)
    new_str2 = re.sub(r'[^\uAC00-\uD7A3a-zA-Z0-9\s]', ' ', new_str)
    new_str3 = re.split(' ', new_str2)
    new_str4 = ' '.join(filter(None, new_str3))
    prefer2.append(new_str4)
    # print(f'{idx}. {prefer2[idx]}')
prefer2_df = pd.DataFrame({'prefer': prefer2})


# Kor, Eng 분리
eng_list = []
kor_list = []
for idx in range(0, df.shape[0]):
    text = prefer2_df.prefer[idx]
    eng_text = ' '.join(re.findall(r'[A-Za-z]+', text))
    kor_text = re.sub(r'[A-Za-z]+', '', text).strip()
    eng_list.append(eng_text)
    kor_list.append(kor_text)


# 영어 띄어쓰기 기준으로 split
eng_split_list = []
for eng in eng_list:
    spl = eng.split()
    eng_split_list.append(spl)
# for idx, eng in enumerate(eng_split_list):
    # print(f'{idx}. {eng}')


# 영어 소문자로 통일
eng_lower = []
for sub in eng_split_list:
    sublist = []
    for eng in sub:
        eng_ = eng.lower()
        sublist.append(eng_)
    eng_lower.append(sublist)
# for idx, eng in enumerate(eng_lower):
    # print(f'{idx}. {eng}')
# print(len(eng_lower))


# 텍스트 파일 불러오기
with open(code_path + 'Postprocessor_Custom_Nouns.txt', 'r', encoding='utf-8-sig') as file:
    a = file.read()
    spl = a.split(',')
    custom_nouns = list()
    for i in spl:
        spl_ = i.strip(" '")
        custom_nouns.append(spl_)
    # print(custom_nouns)

with open(code_path + 'Postprocessor_Replace.txt', 'r', encoding='utf-8-sig') as file:
    b = file.read()
    spl = b.split(',')
    replace = dict()
    for i in spl:
        key_val = i.split(':')
        key = key_val[0].strip(" '")
        val = key_val[1].strip(" '")
        replace[key] = val
    # print(replace)
    
with open(code_path + 'Postprocessor_Stopwords.txt', 'r', encoding='utf-8-sig') as file:
    c = file.read()
    spl = c.split(',')
    stopwords = set()
    for i in spl:
        spl_ = i.strip(" '")
        stopwords.add(spl_) 
    # print(stopwords)


okt = Okt()

# 명사 추가
custom_nouns = custom_nouns
for noun in custom_nouns:
    okt.add_dictionary(noun, 'Noun')

# 단어 치환
replace = replace

# 불용어 처리
stopwords = stopwords

postprocessor = Postprocessor(
    base_tagger=okt,
    passtags={'Noun'},
    replace=replace,
    stopwords=stopwords,
    # passwords=passwords,
    # ngrams=ngrams
)

results = []
for idx in range(0, df.shape[0]):
    elem = kor_list[idx]
    # print(postprocessor.pos(elem))
    post_onechar = postprocessor.pos(elem)
    result = []
    for word, tag in post_onechar:
        if len(word) > 1:
            result.append(word)
    results.append(result)
    # print(f'{idx}. {result}')


# 리스트 중복 요소 제거
Prefer_kor_final = []
for idx in range(0, df.shape[0]):
    a = set(results[idx])
    b = list(a)
    Prefer_kor_final.append(b)
print(Prefer_kor_final)
# print(len(Prefer_kor_final))


# 데이터프레임 생성
Prefer_kor_final_df = pd.DataFrame({'Preference_KOR': Prefer_kor_final})
Prefer_kor_final_df.Preference_KOR = Prefer_kor_final_df.Preference_KOR.apply(lambda x: ', '.join(x))
Prefer_kor_final_df.to_csv(csv_path + 'Preference_Kor.csv', index=False)

c:\Users\LG\anaconda3\envs\crawling\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[['데이터 분석', '머신러닝', '기상 데이터', '데이터 모델링'], ['머신러닝', '블록체인', '커뮤니케이션', '빅데이터 분석', '대회 수상자', '리포트 작성', '통계분석', '암호화폐', '영어 커뮤니케이션'], ['머신러닝', '블록체인', '커뮤니케이션', '빅데이터 분석', '대회 수상자', '리포트 작성', '통계분석', '암호화폐', '영어 커뮤니케이션'], ['통계분석', '전산 지식', '데이터 분석'], ['데이터 시각화', '클라우드 경험', '프레임워크'], ['개발경험', '인공지능', '자율주행 데이터', '이미지 데이터', '데이터 분석', '레이더 데이터', '컴퓨터공학', '통계분석'], ['머신러닝', '클라우드 경험', '통계학과', '논리적 사고', '모델 개발', '데이터 분석', '영어 커뮤니케이션', '수학과', '데이터 모델링', '데이터 사이언티스트', '컴퓨터과학'], ['문제 정의 및 해결', '가설 수립 및 검증', '데이터 분석', '구조적 사고', '데이터 모델링'], ['머신러닝', '최신 기술 연구', '통계분석', '모델 개발', '테스트 설계 분석', '이슈 해결', '데이터 마이닝', '모바일 데이터', '고객 분석', '데이터 분석', '정형 데이터', '적극적', '데이터 모델링', '커뮤니케이션', '데이터 사이언스', '통계적 지식', '도메인 지식', '문제 정의 및 해결', '주도적'], ['논문 구현', '머신러닝', '인공지능', '데이터 분석', '통계분석'], ['도메인 지식', '머신러닝', '논문 구현', '상용 서비스 구현', '커뮤니케이션', '모델 개발 및 배포', '영어 커뮤니케이션', '실무경험', '통계분석', '데이터 마이닝', '모델 개발'], ['통계적 지식', '경영학', '프레임워크', '프로젝트 경험', '경영정보학', '데이터 분석', '딥러닝', '경제학', '컴퓨터공학', '프로그래밍 언어', '산업공학'], ['도메인 지식', '최신